In [1]:
!pip install deepeval


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")

# Initialize Python porter stemmer
ps = PorterStemmer()
def lemmatize(sent):
    return [ps.stem(word) for word in sent.split()]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rishikasrinivas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
import fitz,os
def read_pdf(pdf_file):
    start=False
    sentences=[]
    start_idx=0
    with fitz.open(pdf_file) as pdf_file:
        for page_index, page in enumerate(pdf_file):
            text = page.get_text("text").lower()
            text=text.split(". ")
            sentences.extend(text)
                
    return sentences
def read_files(root_dir):
    
    lines=[]
    for files in os.listdir(root_dir):
        if files[-4:] != '.pdf':
            continue
        sentences = read_pdf(f"{root_dir}/{files}")
        lines.extend(sentences)


    return lines

from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')
def vec(sentences):
    # Encode sentences
    embeddings = model.encode([sentences[0], sentences[1]])
    
    # Compute cosine similarity
    similarity = util.cos_sim(embeddings[0], embeddings[1])
    return similarity.item() # Value close to 1 indicates high similarity
sentences = read_files("Docs")
def find(target, src):
    found=False
    matching_sentence=""
 
    for idx,sentence in enumerate(src):
        pred=" ".join(lemmatize(target))
        test=" ".join(lemmatize(sentence))
        cos = vec([pred,test])
        if pred in test or cos > 0.7:
            
            found=True
            st_idx=idx
            matching_sentence=sentence
            return found, matching_sentence, 
            
    return found, matching_sentence
    

In [20]:
found=0
def find_in_hand(hand,target):
    max_=-1.0
    for row in hand.iterrows():
        hand_rel = f" {row[1]['subj']} {row[1]['rel']} {row[1]['obj']}"
        sub_h = row[1]['subj']
        cos=vec([hand_rel, target])
        

        if cos > max_:
            max_ = cos
            best=hand_rel
        
        cos=vec([best, hand_rel])
        if sub_h.lower() not in best.lower() or cos < 0.7:
            print(f"Closest match for {hand_rel} is {best} with {cos}")
            continue
        else:
            return best
            #print(f"Closest match for {hand_rel} is {best} with {vec([best, hand_rel])}")
            #print(rouge_([hand_rel], [best]))
            
    
            

In [24]:
from deepeval.metrics import ContextualRecallMetric
from deepeval.test_case import LLMTestCase
import pandas as pd
hand = pd.read_csv("Annotations.csv")
preds = pd.read_csv("NewRels_Skip3.csv") 

def test_case(output, true, sentence):
    return LLMTestCase(
        input=f"Extract relationships from {sentence}",
        actual_output=output,
        # Expected output is the "ideal" output of your LLM, it is an # extra parameter that's needed for contextual metrics
        expected_output=true,
        retrieval_context=sentences
    )
metric = ContextualRecallMetric(threshold=0.5)
score=0
for p in preds.iterrows():
    ref = p[1]['ref']
    rel = p[1]['rel']
    subj = p[1]['subj']
    obj = p[1]['obj']
    out=f"{subj} {rel} {obj}"
    hand_match=find_in_hand(hand,out)
    
    found, match = find(out, sentences)
    if not found:
        continue
    idx=sentences.index(match)
    test=test_case(out, hand_match, [sentences[i] for i in range(idx-3, idx+4)])
    metric.measure(test)
    score += metric.score
    print(metric.reason)
    print(metric.is_successful())

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

The score is 1.00 because all sentences in the expected output are perfectly supported by the nodes in the retrieval context, indicating a flawless match. Great job!
True


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

The score is 1.00 because the expected output aligns perfectly with the information in the retrieval context. Fantastic job!
True


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

The score is 1.00 because every sentence in the expected output perfectly matches with information from the nodes in the retrieval context. Great job!
True


In [29]:
print(f"Recall score: {100*score/len(preds)}%")

Recall score: 80.50314465408806%
